In [2]:
# Author: noe.sturm@novartis.com

In [3]:
import os, sys
import numpy as np
import pandas as pd
import importlib
from scipy.io import mmread
sys.path.append('/path/to/repo/performance_evaluation/development_code/')
import modeval


In [4]:
importlib.reload(modeval) # dev stuff

<module 'modeval' from '/home/sturmno1/Projects/Codes/repos/performance_evaluation/development/modeval.py'>

In [6]:
result_dir = 'results_sparsechem/'
model_dir = 'models/'

In [7]:
result_dir = '/db/melloddy/users_workspace/sturmno1/single_partner_studies/public_trivial_model/results/'
model_dir = '/db/melloddy/users_workspace/sturmno1/single_partner_studies/public_trivial_model/models/'

This demonstration is based on a toy HP grid search made with the chembl dataset and trivial LSH key.<br>
For the purpose of this demonstration, the HP search was restrained to a 10 epochs and searched a small number of hyperparameters

### Get the best hyperparameter settings from metrics.csv files

In [9]:
# collect performance metrics from metrics file. 
# this assumes metrics files are named after the hyperparameters (see below into details of perf_from_metrics() )
metrics_df = modeval.perf_from_metrics(result_dir, verbose=True)

# Disclaimer: this does not consider the --min_samples criteria, all tasks are considered!

# melt 
metrics_dfm = modeval.melt_perf(metrics_df, perf_metrics=['auc_pr_va', 'auc_va', 'max_f1_va', 'kappa_va', 'avg_prec_va'])

# Find out the best hyperparameters: one row per score type gives the best HP per score type
modeval.best_hyperparam(metrics_dfm)

Loaded 30 metrics files


,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,score_type,value
17,2000,0.6,1e-05,0.001,5,10,auc_pr_va,0.715337
0,2000,0.6,1e-05,0.001,5,10,auc_va,0.770708
12,2000,0.6,1e-05,0.001,5,10,avg_prec_va,0.735341
24,2000,0.6,1e-05,0.001,5,10,kappa_va,0.285665
4,2000,0.6,1e-05,0.001,5,10,max_f1_va,0.758460


### Get the best hyperparameter settings from conf.npy file

In [12]:
# collect performance metrics from conf file 
conf_df = modeval.perf_from_conf(model_dir)

# Disclaimer: this does not consider the --min_samples criteria, all tasks are considered!

# melt: in conf files, there is only auc_pr and auc_roc (no kappa, f1, ...)
conf_dfm = modeval.melt_perf(conf_df, perf_metrics=['auc_pr_va', 'auc_va']) 

# Find out the best hyperparameters
modeval.best_hyperparam(conf_dfm)

,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,score_type,value
6,10,2000,0.6,0.00001,0.001,5,auc_pr_va,0.715337
0,10,2000,0.6,0.00001,0.001,5,auc_va,0.770708


### Let's dive into the functions used here

There are two entry points to collect performance results from sparsechem:
- results/*-metrics.csv files 
- models/*-conf.npy files

Here we will fetch performance reports from both entry points and see the specifities of each: 

### 1/ Metrics files

In [13]:
# 1/ load the performance metrics from the *metrics.csv file in "results" folder
# 
# modeval.perf_from_metrics() --> loads performance reports from metrics files in result_dir including  
# It assumes the metrics filenames contain information about hyperparameters. 
# It extracts the hyperparameters settings present in the filename and adds columns to perf metrics report
# columns names of hyperparamters are prefixed with "_hp"
# 
# Ultimately: fetching hyperparameters from filenames is not very good practice and should be mitigated in the future
# Ideally: sparsechem should provide one performance report containing all numbers including HPs
# 


perf_metrics = modeval.perf_from_metrics(result_dir, verbose=True)
perf_metrics
# => is a dataframe containing one row per task, one column per perf metrics + columns containing metadata of models/task (e.g. HPs, num_pos, valid fold,.. )
# NB: the dtaframe contains a row for all tasks. No exceptions such as minimum number of samples of each class. 
#     ==> This assumes filtering of tasks with less than X positives and X negatives should be done manually


Loaded 30 metrics files


,task,num_pos,num_neg,num_pos_va,num_neg_va,auc_tr,auc_va,auc_pr_tr,auc_pr_va,avg_prec_tr,...,kappa_va,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,fold_test,model
0,0,26,28,5,1,0.991182,0.600000,0.989486,0.918333,0.989719,...,0.076923,1600,0.6,1e-05,0.001,5,10,2,None,Y
1,1,25,28,2,3,0.965217,0.333333,0.961833,0.291667,0.962680,...,-0.363636,1600,0.6,1e-05,0.001,5,10,2,None,Y
2,2,25,27,3,4,0.911067,0.333333,0.923706,0.549206,0.925274,...,0.000000,1600,0.6,1e-05,0.001,5,10,2,None,Y
3,3,28,29,5,2,0.913043,0.900000,0.862808,0.963333,0.867996,...,0.461538,1600,0.6,1e-05,0.001,5,10,2,None,Y
4,4,29,27,22,25,1.000000,0.285455,1.000000,0.385190,1.000000,...,-0.042662,1600,0.6,1e-05,0.001,5,10,2,None,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106405,3542,6353,5695,1389,1248,0.995582,0.930653,0.996392,0.938970,0.996392,...,0.696039,1200,0.6,1e-05,0.001,5,10,4,None,Y
106406,3543,4038,8010,862,1775,0.995366,0.937612,0.991862,0.901494,0.991863,...,0.678433,1200,0.6,1e-05,0.001,5,10,4,None,Y
106407,3544,2308,9740,495,2142,0.995813,0.941757,0.984444,0.830207,0.984448,...,0.595842,1200,0.6,1e-05,0.001,5,10,4,None,Y
106408,3545,8736,3312,1934,703,0.993217,0.928366,0.997462,0.970711,0.997462,...,0.676348,1200,0.6,1e-05,0.001,5,10,4,None,Y


In [14]:
perf_metrics.shape # nrows = ntasks * 5cv * n_hyperparameters

(106410, 24)

In [15]:
perf_metrics.columns

Index(['task', 'num_pos', 'num_neg', 'num_pos_va', 'num_neg_va', 'auc_tr',
       'auc_va', 'auc_pr_tr', 'auc_pr_va', 'avg_prec_tr', 'avg_prec_va',
       'max_f1_tr', 'max_f1_va', 'kappa_tr', 'kappa_va', 'hp_hidden_sizes',
       'hp_last_dropout', 'hp_weight_decay', 'hp_learning_rate',
       'hp_learning_steps', 'hp_epochs', 'fold_va', 'fold_test', 'model'],
      dtype='object')

In [21]:
# by default: number of folds is 5 (n_cv=5) and the argument verify is False
# however turning verify to True allows checking if for any of the tasks there are less than n_cv metrics reprots 
# this can be very usefull in case of extensive hyperparameter grid search to identify failed jobs. 

perf_metrics = modeval.perf_from_metrics(result_dir, verbose=True, verify=True)

# => by default, n_cv=5, the function expects folds 0,1,2,3,4 being run. 
#for the purpose of this demonstration, one result/model was removed from folder. See the warning message
# column on the far right states which folds were run, one can see fold 0 is missing

Loaded 29 metrics files
Fold runs found :
 hp_hidden_sizes  hp_last_dropout  hp_weight_decay  hp_learning_rate  hp_learning_steps  hp_epochs
1200             0.6              1e-05            0.001             5                  10           0,1,2,3,4
1200.1200        0.6              1e-05            0.001             5                  10           0,1,2,3,4
1600             0.6              1e-05            0.001             5                  10           0,1,2,3,4
1600.1600        0.6              1e-05            0.001             5                  10           0,1,2,3,4
2000             0.6              1e-05            0.001             5                  10           0,1,2,3,4
800              0.6              1e-05            0.001             5                  10             1,2,3,4
Name: fold_va, dtype: object


In [22]:
# If desired, one can also fetch perofrmance results for a subset of the tasks using the argument "tasks_for_eval":
# using task indices (columns in mtx files), it is possible to mask out some tasks
# i.e. let's fetch metrics for tasks 1, 23, 124


modeval.perf_from_metrics(result_dir, verbose=True, tasks_for_eval=[1,23,124])



Loaded 29 metrics files


,task,num_pos,num_neg,num_pos_va,num_neg_va,auc_tr,auc_va,auc_pr_tr,auc_pr_va,avg_prec_tr,...,kappa_va,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,fold_test,model
0,1,25,28,2,3,0.965217,0.333333,0.961833,0.291667,0.962680,...,-0.363636,1600,0.6,1e-05,0.001,5,10,2,None,Y
1,23,2857,3225,569,646,0.983075,0.784332,0.983510,0.798325,0.983513,...,0.401895,1600,0.6,1e-05,0.001,5,10,2,None,Y
2,124,18158,23678,3821,5051,0.918262,0.606837,0.897007,0.539238,0.897012,...,0.127662,1600,0.6,1e-05,0.001,5,10,2,None,Y
3,1,25,28,1,2,0.943910,0.500000,0.949771,0.250000,0.950684,...,0.000000,800,0.6,1e-05,0.001,5,10,4,None,Y
4,23,2857,3225,575,675,0.973960,0.799170,0.974674,0.816002,0.974680,...,0.471448,800,0.6,1e-05,0.001,5,10,4,None,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,23,2857,3225,575,675,0.987840,0.800600,0.987947,0.819158,0.987950,...,0.472464,2000,0.6,1e-05,0.001,5,10,4,None,Y
83,124,18158,23678,3520,4596,0.926390,0.622052,0.907531,0.550745,0.907535,...,0.144137,2000,0.6,1e-05,0.001,5,10,4,None,Y
84,1,25,28,1,2,0.958333,1.000000,0.951286,1.000000,0.952450,...,0.000000,1200,0.6,1e-05,0.001,5,10,4,None,Y
85,23,2857,3225,575,675,0.980500,0.800433,0.980673,0.817476,0.980677,...,0.477558,1200,0.6,1e-05,0.001,5,10,4,None,Y


### 2/ Config files

In the models/\*.conf.npy files, there are actually two performance reports: 
- individual tasks performance reports
- aggregate performance reports (average over all tasks)

In addition, conf.npy files contain all settings used for training the model. 

#### individual tasks performance 

In [23]:
# 2/ collect the performance scores from the *conf.npy in the "models" folder

# a/ Lets get performance of each individual task. 
perf_conf = modeval.perf_from_conf(model_dir, aggregate=False)
perf_conf

# this will lead into a similar data frame compared to modeval.perf_from_metrics(), one row per task, one columns per metrics/metadatas
# Hyperparamter column names are prefixed by "hp_" 

# NB: performance are reported for all tasks, however not all the metrics are present in conf files (see columns names below)


,task,fold_te,fold_va,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,n_tasks_eval,min_samples,auc_va,auc_pr_va,model
0,0,None,3,10,1200,0.6,0.00001,0.001,5,3547,50,0.750000,0.731250,Y
1,1,None,3,10,1200,0.6,0.00001,0.001,5,3547,50,1.000000,1.000000,Y
2,2,None,3,10,1200,0.6,0.00001,0.001,5,3547,50,0.750000,0.633333,Y
3,3,None,3,10,1200,0.6,0.00001,0.001,5,3547,50,0.793651,0.691927,Y
4,4,None,3,10,1200,0.6,0.00001,0.001,5,3547,50,NaN,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102858,3542,None,2,10,800,0.6,0.00001,0.001,5,3547,50,0.919514,0.918624,Y
102859,3543,None,2,10,800,0.6,0.00001,0.001,5,3547,50,0.925979,0.869812,Y
102860,3544,None,2,10,800,0.6,0.00001,0.001,5,3547,50,0.932107,0.816825,Y
102861,3545,None,2,10,800,0.6,0.00001,0.001,5,3547,50,0.923566,0.968926,Y


In [38]:
perf_conf.shape

(3547, 14)

In [41]:
# columns in perf from metrics file: max_f1, kappa, avg_prec are here
perf_metrics.columns

Index(['task', 'num_pos', 'num_neg', 'num_pos_va', 'num_neg_va', 'auc_tr',
       'auc_va', 'auc_pr_tr', 'auc_pr_va', 'avg_prec_tr', 'avg_prec_va',
       'max_f1_tr', 'max_f1_va', 'kappa_tr', 'kappa_va', 'hp_hidden_sizes',
       'hp_last_dropout', 'hp_weight_decay', 'hp_learning_rate',
       'hp_learning_steps', 'hp_epochs', 'fold_va', 'fold_test', 'model'],
      dtype='object')

In [40]:
# columns in perf from metrics file: max_f1, kappa, avg_prec are NOT here!
# no perf metrics are reported on training set because function does not fetch results for training
# POSSIBLE ADD ON: could be possible to add training metrics
perf_conf.columns

Index(['task', 'fold_te', 'fold_va', 'hp_epochs', 'hp_hidden_sizes',
       'hp_last_dropout', 'hp_weight_decay', 'hp_learning_rate',
       'hp_learning_steps', 'n_tasks_eval', 'min_samples', 'auc_va',
       'auc_pr_va', 'model'],
      dtype='object')

In [36]:
# Similarly to modeval.perf_from_metrics(), if desired, one can also fetch perofrmance results for a subset of the tasks using the argument "tasks_for_eval":
# using task indices (columns in mtx files), it is possible to mask out some tasks
# i.e. let's fetch metrics for tasks 1, 23, 123

modeval.perf_from_conf(model_dir, aggregate=False, tasks_for_eval=[1,23,123])


,task,fold_te,fold_va,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,n_tasks_eval,min_samples,auc_va,auc_pr_va,model
0,1,None,0,20,400,0.2,0.0,0.001,10,3,25,0.672727,0.361825,Y
1,23,None,0,20,400,0.2,0.0,0.001,10,3,25,0.805453,0.812700,Y
2,123,None,0,20,400,0.2,0.0,0.001,10,3,25,0.690300,0.134965,Y


#### Aggregate performance report

In [43]:
# b/ we can also directly load performance aggregates (averaged over the tasks) from the conf file
# this is done turing the argument "aggregate" to True

perf_conf_agg = modeval.perf_from_conf(model_dir, aggregate=True)
perf_conf_agg

# the function actually fetches the aggregate performance report from the conf file and does not the aggregation iteself. 
# Aggregation is done by sparsechem/ 

# ! NB: here the aggregation considers only tasks verifying the --min_sample option 
# that is, if --min_sample is in use with N_MIN=50, this will report a different result that if the aggrgation is done over all tasks

,fold_te,fold_va,hp_epochs,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,min_samples,auc_va_mean,auc_pr_va_mean,max_f1_va_mean,kappa_va_mean,train_time_1epochs,model
0,None,0,20,400,0.2,0.0,0.001,10,25,0.770633,0.71674,0.757492,0.303348,12.306314,Y


,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,variable,value
2,400,0.2,1e-05,0.001,10,20,auc_pr_va,0.718885
1,400,0.2,1e-05,0.001,10,20,avg_prec_va,0.737953
3,400,0.2,1e-05,0.001,10,20,kappa_va,0.295114
0,400,0.2,1e-05,0.001,10,20,max_f1_va,0.758979


In [62]:
# metling can also be done with perf_metrics, but need to specify metrics columns
modeval.melt_perf(perf_metrics, perf_metrics=['auc_pr_va'])

,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,variable,value
28376,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.823777
28377,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.458889
28378,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.579167
28379,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.813080
28380,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,NaN
...,...,...,...,...,...,...,...,...,...
31918,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.939423
31919,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.898452
31920,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.834398
31921,400,0.2,1e-05,0.001,10,20,0,auc_pr_va,0.974911


### Functions to manipulate performance metrics data frames: 

In [50]:
# aggregate performance over folds
modeval.aggregate_fold_perf?

Signature: modeval.aggregate_fold_perf(metrics_df, min_samples, n_cv=5, verify=True)<br>
Docstring:<br>
HP performance aggregation over folds. <br>
    From the metrics dataframe yielded by perf_from_metrics(), does the aggregation over the fold (mean, std) results in one perf per fold.<br>
\#     :param pandas df metrics_df: metrics dataframe yielded by perf_from_metrics() <br>
\#     :param int min_sample: minimum number of each class (overal) to be considered in mean<br>
\#     :param int n_cv: number of folds to look for<br>
\#     :param bool verify: checks for missing folds runs in CV and prints a report if missing jobs<br>
\#     :return dtype: pandas df containing performance per task aggregated over each fold<br>
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py<br>
Type:      function<br>

In [53]:
modeval.aggregate_fold_perf(perf_metrics, min_samples=50, n_cv=1)
# results in one row per fold. Obviousely, given the toy data was only run over one fold, we obtain only one row. 

,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,fold_va,auc_tr_mean,auc_va_mean,auc_pr_tr_mean,...,auc_tr_stdev,auc_va_stdev,auc_pr_tr_stdev,auc_pr_va_stdev,avg_prec_tr_stdev,avg_prec_va_stdev,max_f1_tr_stdev,max_f1_va_stdev,kappa_tr_stdev,kappa_va_stdev
0,400,0.2,1e-05,0.001,10,20,0,0.975565,0.777175,0.956839,...,0.023477,0.138973,0.069231,0.237505,0.06889,0.23073,0.071605,0.19585,0.1801,0.250562


In [55]:
# aggregate performance over all dataframe
modeval.aggregate_overall?

Signature: modeval.aggregate_overall(metrics_df, min_samples)
Docstring:
HP performance aggregation overall . 
    From the metrics dataframe yielded by perf_from_metrics(), does the aggregation over the CV (mean, std) results in one perf hyperparameter.
#     :param pandas df metrics_df: metrics dataframe yielded by perf_from_metrics() 
#     :param int min_sample: minimum number of each class (overal) to be considered in mean
#     :return dtype: pandas df containing performance per hyperparameter setting
    
File:      ~/Projects/Codes/repos/performance_evaluation/development/modeval.py
Type:      function


Signature: modeval.aggregate_overall(metrics_df, min_samples)<br>
Docstring:<br>
HP performance aggregation overall . <br>
    From the metrics dataframe yielded by perf_from_metrics(), does the aggregation over the CV (mean, std) results in one perf hyperparameter.<br>
\#     :param pandas df metrics_df: metrics dataframe yielded by perf_from_metrics() <br>
\#     :param int min_sample: minimum number of each class (overal) to be considered in mean<br>
\#     :return dtype: pandas df containing performance per hyperparameter setting<br>
    
File:      ~/Projects/Codes/repos/ml_tools/modeval.py
Type:      function

In [56]:
modeval.aggregate_overall(perf_metrics, min_samples=50)

# produces one row for all 
# performance contains mean metrics + standard deviations.. 

,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,auc_tr_mean,auc_va_mean,auc_pr_tr_mean,auc_pr_va_mean,...,auc_tr_stdev,auc_va_stdev,auc_pr_tr_stdev,auc_pr_va_stdev,avg_prec_tr_stdev,avg_prec_va_stdev,max_f1_tr_stdev,max_f1_va_stdev,kappa_tr_stdev,kappa_va_stdev
0,400,0.2,1e-05,0.001,10,20,0.975565,0.777175,0.956839,0.691453,...,0.023477,0.138973,0.069231,0.237505,0.06889,0.23073,0.071605,0.19585,0.1801,0.250562


In [57]:
modeval.aggregate_overall(perf_metrics, min_samples=50).columns

Index(['hp_hidden_sizes', 'hp_last_dropout', 'hp_weight_decay',
       'hp_learning_rate', 'hp_learning_steps', 'hp_epochs', 'auc_tr_mean',
       'auc_va_mean', 'auc_pr_tr_mean', 'auc_pr_va_mean', 'avg_prec_tr_mean',
       'avg_prec_va_mean', 'max_f1_tr_mean', 'max_f1_va_mean', 'kappa_tr_mean',
       'kappa_va_mean', 'auc_tr_stdev', 'auc_va_stdev', 'auc_pr_tr_stdev',
       'auc_pr_va_stdev', 'avg_prec_tr_stdev', 'avg_prec_va_stdev',
       'max_f1_tr_stdev', 'max_f1_va_stdev', 'kappa_tr_stdev',
       'kappa_va_stdev'],
      dtype='object')

In [59]:
# aggregate performance over folds --> get average per task
modeval.aggregate_task_perf(perf_metrics, min_samples=50)
# produce one row for each task, averaged over the folds. 
# performance contains mean metrics + standard deviations.. 
# here obviousely since toy data is only one fold run, it does not do much.

Fold runs found :
 hp_hidden_sizes  hp_last_dropout  hp_weight_decay  hp_learning_rate  hp_learning_steps  hp_epochs
400              0.2              1e-05            0.001             10                 20           0
Name: fold_va, dtype: object


,hp_hidden_sizes,hp_last_dropout,hp_weight_decay,hp_learning_rate,hp_learning_steps,hp_epochs,task,num_pos,num_neg,num_pos_va_mean,...,auc_tr_stdev,auc_va_stdev,auc_pr_tr_stdev,auc_pr_va_stdev,avg_prec_tr_stdev,avg_prec_va_stdev,max_f1_tr_stdev,max_f1_va_stdev,kappa_tr_stdev,kappa_va_stdev
0,400,0.2,1e-05,0.001,10,20,8,1444,1369,324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,400,0.2,1e-05,0.001,10,20,9,398,2415,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,400,0.2,1e-05,0.001,10,20,10,79,2734,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,400,0.2,1e-05,0.001,10,20,12,2857,3225,612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,400,0.2,1e-05,0.001,10,20,13,393,5689,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,400,0.2,1e-05,0.001,10,20,3542,6353,5695,1322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1504,400,0.2,1e-05,0.001,10,20,3543,4038,8010,845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1505,400,0.2,1e-05,0.001,10,20,3544,2308,9740,472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1506,400,0.2,1e-05,0.001,10,20,3545,8736,3312,1756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
